# Tools

Some LLMs are capable of acknowledging when to use tools and how to use them. For example, the Ollama tools that are compatible with tool calling can be found in this web page:

https://ollama.com/search?c=tools

In this tutorial we are goind to examine how the models use (or not) tools, how tools can optimize their capabilities and how to properly design tools that have actual impact.

An LLM has been trained with data that reach up to a specific date in time and on a broad context of information. Tools enable them to get up-to-date information and receive specialized knowledge that was possibly not available to them during training.

In this tutorial we will use DuckDuckGo search engine to enable an LLM to get up-to-date information. We will also show how a Wikipedia search can enhance details of retrieved information.

### Load model

First, let's start by loading a model that is compatible with tools.

In [1]:
from langchain_ollama import ChatOllama

# Initialize the ChatOllama model with the specified model name
model_name = 'qwen3-vl:4b'

# and initialize the ChatOllama instance
chat_model = ChatOllama(
    model=model_name,
    validate_model_on_init=True,
    temperature=0.7
)

## Search tool

I this section we will have a look at the DuckDuckGo search engine and what it returns. First let's try it out.

In this tool we can submit queries and get snippets (summaries + titles) fromt some top results (like the snippets we get from results when we search in Google).

In [2]:
from langchain_community.tools import DuckDuckGoSearchRun

ddg_search = DuckDuckGoSearchRun()

search_result = ddg_search.run('Who is the president of Venezuela?')

# This tool returns a string with the snippets of the search result
print(type(search_result))
print(search_result)

<class 'str'>
1 week ago - On 3 January 2026, the United States captured then-president Nicolás Maduro, as well as his wife Cilia Flores, and extracted them from the country. As a result, Delcy Rodríguez became the acting president. Rodríguez is the current acting president. As a self described republic with a ... 14 hours ago - Nicolás Maduro Moros (born 23 November 1962) is a Venezuelan politician and former union leader who became the president of Venezuela in 2013. On 3 January 2026, US forces captured Maduro and his wife, Cilia Flores; they were transported to the US and charged with drug trafficking to which ... 2 days ago - Maduro's government has accused the United States of organizing a coup d'état to remove him and take control of the country's oil reserves. Guaidó rejects the characterization of his actions as a coup, saying that his movement is backed by peaceful volunteers. As of June 2019, Guaidó was recognized as the acting President of Venezuela by 54 countries. 1 week 

## LangChain tool

To make a LangChain tool, we need to decorate a python function and let it return a value that the LLM can handle and integrate into its chat logic. Some things are important to notice here:

- The functions need to have a `@tool` decoration for LangChain to register them as such.
- The pydantic definition of functions are important for informing the LLM what the tool actually does. I.e., it is better to have tool function in this form `def foo(x: int) -> float:` rather than in this form `def foo(x):`.
- Comments in the tool function definition are important, because they allow the model to understand what the model does. Additionally, the comments need to be added in the *docstring* way so that LangChain can include them properly inside a structured schema that is injected in the LLM.

In [3]:
from langchain.tools import tool

In [4]:
@tool
def search(query: str) -> str:
    """Search for information on the internet using a search tool."""
    return ddg_search.run(query)

### With vs without the tool

Let's see how the model responds with and without the tool. For this, we will keep the version of the model without the tool and create a new instance that has the tool binded.

In [5]:
model_with_tool = chat_model.bind_tools([search])

In [6]:
result_no_tool = chat_model.invoke("Who is the President of Venezuela?")

In [7]:
print(result_no_tool.content)

As of **July 2024**, the **de facto leader of Venezuela is Nicolás Maduro**, who has been the President since **2013**. However, the situation is highly contested due to the **political crisis and ongoing conflict** between Maduro's government and the opposition.

### Key Details:
1. **Official Status**:  
   - Maduro was **elected in 2013** and re-elected in **2018** (though the 2018 election was widely disputed).  
   - In **2023**, he won a controversial presidential election, which the opposition and many international observers deemed fraudulent.  
   - The **National Assembly** (controlled by Maduro's allies) declared a **"state of emergency"** in May 2023, suspending the presidency and forcing Maduro to step down temporarily. However, he **was re-elected in a disputed vote** later that year, and the National Assembly reinstated him.  

2. **International Recognition**:  
   - **United Nations, Brazil, Colombia, and other countries** recognize Maduro as the legitimate president. 

In [8]:
from langchain_core.messages import HumanMessage, ToolMessage

# We will use a HumanMessage and a ToolMessage for keeping better structure.

messages = [
    HumanMessage(
        content="You must use the search tool to answer this question: "
                "Who is the president of Venezuela?"
    )
]

# First turn
ai_msg = model_with_tool.invoke(messages)
messages.append(ai_msg)

# Tool execution
if ai_msg.tool_calls:
    for tool_call in ai_msg.tool_calls:
        print(tool_call)
        # Corrected: Access 'args' as a dictionary key and then 'query' within it
        tool_result = search.invoke(tool_call['args']['query'])
        messages.append(
            ToolMessage(
                content=str(tool_result),
                tool_call_id=tool_call['id'] # Access 'id' as a dictionary key as well
            )
        )

    # New human turn (required!)
    messages.append(
        HumanMessage(
            content="Using the tool results above, answer the question."
        )
    )

    final_response = model_with_tool.invoke(messages)
    print(final_response.content)
else:
    print("Model did not call tool:")
    print(ai_msg.content)

{'name': 'search', 'args': {'query': 'current president of Venezuela'}, 'id': '2b0f7d3c-a61d-4c41-8f2d-dfb4103755db', 'type': 'tool_call'}
Based on the provided search results, Delcy Rodríguez is currently serving as the acting president of Venezuela. This is indicated in multiple recent entries (e.g., "2 days ago" and "3 days ago"), which state that she was sworn in as interim president following the capture of Nicolás Maduro on January 3, 2026. While the date of 2026 appears to be a future event in the search results, the tool responses consistently identify Rodríguez as the acting president in this hypothetical scenario. Therefore, according to the given information, **Delcy Rodríguez** is the acting president of Venezuela.


### Tool result

We see above the the model decided to use the tool and pass as search query the term "current president of Venezuela". This shows that the model internally could resolve what information it needed to retrieved based on the human input.

In [9]:
from langchain_community.retrievers import WikipediaRetriever

In [10]:
retriever = WikipediaRetriever()

In [11]:
docs = retriever.invoke("Kasawari")

In [12]:
print(type(docs))
print(type(docs[0]))
print(docs[0].page_content)

<class 'list'>
<class 'langchain_core.documents.base.Document'>
Cassowaries (Indonesian: kasuari; Biak: man suar 'bird strong'; Tok Pisin: muruk; Papuan: kasu weri 'horned head') are flightless birds of the genus Casuarius, in the order Casuariiformes. They are classified as ratites, flightless birds without a keel on their sternum bones. Cassowaries are native to the tropical forests of New Guinea (Western New Guinea and Papua New Guinea), the Moluccas (Seram and Aru Islands), and northeastern Australia.
Three cassowary species are extant. The most common, the southern cassowary, is the third-tallest and second-heaviest living bird, smaller only than the ostrich and emu. The other two species are the northern cassowary and the dwarf cassowary; the northern cassowary is the most recently discovered and the most threatened. A fourth, extinct, species is the pygmy cassowary.
Cassowaries are very wary of humans, but if provoked, they are capable of inflicting serious, even fatal, injuries

In [34]:
@tool
def wikipedia_search(query: str) -> str:
    """Search for information on wikipedia using its search tool."""
    docs = retriever.invoke(query)
    return docs[0].page_content

In [35]:
model_with_tool = chat_model.bind_tools([search,wikipedia_search])

In [16]:
result_no_tool = chat_model.invoke("How many Kasawari species are there?")

In [17]:
print(result_no_tool.content)

The term **"Kasawari"** refers to a genus of fish in the family **Serranidae** (which includes groupers, sea basses, and other reef-dwelling fish). However, it's important to clarify the correct taxonomic name:  

- The genus is **Kasawaria**, not "Kasawari" (which is a common misspelling or misnomer).  
- **Kasawaria** is a monotypic genus, meaning it contains **only one species**:  
  **Kasawaria maris** (also known as the "Kasawari" or "Kasawari grouper").  

### Key Details:
- **Taxonomic status**:  
  - Kingdom: Animalia  
  - Phylum: Chordata  
  - Class: Actinopterygii (ray-finned fish)  
  - Order: Serraniformes  
  - Family: Serranidae  
  - Genus: *Kasawaria*  
  - Species: *Kasawaria maris* (validly described as a single species).  

- **Why confusion arises**:  
  - The name "Kasawari" is often used informally for *Kasawaria maris*, but the scientific genus is **Kasawaria**, not "Kasawari."  
  - Some sources may incorrectly use "Kasawari" as the genus name, but this is out

In [37]:
from langchain_core.messages import HumanMessage, ToolMessage

# We will use a HumanMessage and a ToolMessage for keeping better structure.

messages = [
    HumanMessage(
        content="You can use any available tool to answer this question: "
                "How many spicies of Kasawari are there?"
    )
]

# First turn
ai_msg = model_with_tool.invoke(messages)
messages.append(ai_msg)

# Tool execution
if ai_msg.tool_calls:
    for tool_call in ai_msg.tool_calls:
        print(tool_call)
        # Corrected: Access 'args' as a dictionary key and then 'query' within it
        tool_result = search.invoke(tool_call['args']['query'])
        messages.append(
            ToolMessage(
                content=str(tool_result),
                tool_call_id=tool_call['id'] # Access 'id' as a dictionary key as well
            )
        )

    # New human turn (required!)
    messages.append(
        HumanMessage(
            content="Using the tool results above, answer the question."
        )
    )

    final_response = model_with_tool.invoke(messages)
    print(final_response.content)
else:
    print("Model did not call tool:")
    print(ai_msg.content)

{'name': 'search', 'args': {'query': 'how many species of Kasawari are there'}, 'id': '11f93411-6230-411a-97ae-77fe481bd994', 'type': 'tool_call'}
There are three species of cassowaries, which are commonly referred to as Kasawari. The tool response confirms that "all three species of cassowaries are considered as Asia's largest bird since the extinction of the Arabian ostrich." Thus, the number of species is **three**.


### Different results

Depending on the randomness expressed by temperature, the model may go through different routes in answering the question. Below you can some indicative instances where the model was able to produce a proper query (asking about the number of kasawari spieces), get a proper answer and come up with a proper response. Here are some examples:

Used the `wikipedia_search` tool with a general query ("Kasawari") but got right results:
```
{'name': 'wikipedia_search', 'args': {'query': 'Kasawari'}, 'id': '18f1325f-db46-49d9-a337-6c3d26acfa8d', 'type': 'tool_call'}
There are three species of cassowaries, which are also known as "Kasawari" in Indonesian. The family Casuariidae includes these three species:  
1. Southern cassowary (*Casuarius casuarius*)  
2. Northern cassowary (*Casuarius unappendiculatus*)  
3. Dwarf cassowary (*Casuarius bennetti*)  

While the tool response mentions "four surviving members" (including the emu), the emu belongs to a different family (Dromaiidae) and is not part of the cassowary group. Thus, the correct count of *Kasawari* (cassowary) species is **three**.
```

Used the `search` tool with a general search term ("Kasawari") and got wrong results:
```
{'name': 'search', 'args': {'query': 'Kasawari'}, 'id': 'd6043885-5bab-474d-b30f-c85d7b1684dc', 'type': 'tool_call'}
Based on the provided search results, there is no specific information about the number of species of "Kasawari" (which appears to be a common name for a song, street, or cultural reference rather than a biological species). The search results do not mention any taxonomic classification, scientific species count, or biological context for "Kasawari." 

However, if "Kasawari" refers to the **Spotted Dove** (*Columba palumbus*), a bird species known by this name in some regions (e.g., the Philippines), it is a **single species**. But since the search results do not confirm this biological context, the answer cannot be definitively derived from the given data. 

**Conclusion**: The search results do not provide information about the number of *species* of "Kasawari." If this term refers to a biological entity, further clarification or a scientific database would be needed.
```

Used the `search` tool with a more specific search term ("how many species of Kasawari are there") and got correct results:
```
{'name': 'search', 'args': {'query': 'how many species of Kasawari are there'}, 'id': '11f93411-6230-411a-97ae-77fe481bd994', 'type': 'tool_call'}
There are three species of cassowaries, which are commonly referred to as Kasawari. The tool response confirms that "all three species of cassowaries are considered as Asia's largest bird since the extinction of the Arabian ostrich." Thus, the number of species is **three**.
```

In [12]:
import python_weather
import asyncio


@tool
async def get_weather(
    city: str,
    days: int = 3
) -> str:
    """
    Get the current weather and a short forecast for the next few days.

    Use this tool when the user asks about:
    - current weather in a city
    - weather forecast for the next few days and for some hour marks during each day

    Parameters:
    - city: Name of the city (e.g. "Athens", "New York")
    - days: Number of forecast days to include (default: 3, max: 5)
    """
    days = max(1, min(days, 5))

    async with python_weather.Client() as client:
        weather = await client.get(city)

        # Current conditions
        result = [
            f"Current weather in {city}:",
            f"- Temperature: {weather.temperature}°C",
            f"- Description: {weather.description}",
            ""
        ]

        # Daily forecast
        result.append("Forecast:")
        for i, daily in enumerate(weather):
            if i >= days:
                break

            result.append(
                f"- {daily.date}: "
                f"sunlight hours {daily.sunlight}, "
                f"low {daily.lowest_temperature}°C, "
                f"high {daily.highest_temperature}°C"
            )
            for hourly in daily.hourly_forecasts:
                result.append(
                    f"time {hourly.time} "
                    f"precipitation {hourly.precipitation}"
                )

        return "\n".join(result)


In [13]:
weather = await get_weather.coroutine("Rethymno", 3)
print(weather)

Current weather in Rethymno:
- Temperature: 16°C
- Description: Light rain

Forecast:
- 2026-01-21: sunlight hours 7.8, low 10°C, high 15°C
time 00:00:00 precipitation 0.0
time 03:00:00 precipitation 0.0
time 06:00:00 precipitation 0.0
time 09:00:00 precipitation 0.2
time 12:00:00 precipitation 0.2
time 15:00:00 precipitation 0.2
time 18:00:00 precipitation 0.1
time 21:00:00 precipitation 0.0
- 2026-01-22: sunlight hours 7.5, low 12°C, high 16°C
time 00:00:00 precipitation 0.0
time 03:00:00 precipitation 0.0
time 06:00:00 precipitation 0.0
time 09:00:00 precipitation 0.0
time 12:00:00 precipitation 0.3
time 15:00:00 precipitation 0.1
time 18:00:00 precipitation 1.4
time 21:00:00 precipitation 0.4
- 2026-01-23: sunlight hours 7.5, low 13°C, high 16°C
time 00:00:00 precipitation 0.2
time 03:00:00 precipitation 0.1
time 06:00:00 precipitation 0.1
time 09:00:00 precipitation 0.0
time 12:00:00 precipitation 0.0
time 15:00:00 precipitation 0.2
time 18:00:00 precipitation 0.2
time 21:00:00 pr

In [25]:
tools = {'serch': search, 'get_weather': get_weather}

In [24]:
model_with_tool = chat_model.bind_tools([search, get_weather])

In [26]:
messages = [
    HumanMessage(
        content="You must use the search tool to answer this question: "
                "I am planning on visiting Rethymno tomorrow. Should I get my raincoat?"
    )
]

# First turn
ai_msg = model_with_tool.invoke(messages)
messages.append(ai_msg)

# Tool execution
if ai_msg.tool_calls:
    for tool_call in ai_msg.tool_calls:
        print(tool_call)

        tool_name = tool_call["name"]
        tool_args = tool_call["args"]

        tool = tools[tool_name]

        tool_result = tool.invoke(tool_args)
        
        messages.append(
            ToolMessage(
                content=str(tool_result),
                tool_call_id=tool_call['id'] # Access 'id' as a dictionary key as well
            )
        )

    # New human turn (required!)
    messages.append(
        HumanMessage(
            content="Using the tool results above, answer the question."
        )
    )

    final_response = model_with_tool.invoke(messages)
    print(final_response.content)
else:
    print("Model did not call tool:")
    print(ai_msg.content)

{'name': 'get_weather', 'args': {'city': 'Rethymno', 'days': 1}, 'id': '5009a6ed-d627-4e48-b8ad-c7b4928b6443', 'type': 'tool_call'}


NotImplementedError: StructuredTool does not support sync invocation.

In [19]:
import python_weather
import asyncio

In [20]:
weather_client = python_weather.Client()

In [32]:
# Await the weather data to get the forecasts
weather = await weather_client.get("Athens Greece")

In [ ]:
print(type(weather))
print(weather.temperature)
print(weather.precipitation)
print(weather.cloud_cover)
print(weather.country)

<class 'python_weather.forecast.Forecast'>
7
1.8
100
9
Greece


In [46]:
forecasts = weather.daily_forecasts
print(type(forecasts))
print(forecasts)
today = forecasts[0]
print(today.date)
print(today.temperature)
print(today.hourly_forecasts)

<class 'list'>
[<python_weather.forecast.DailyForecast date=datetime.date(2026, 1, 21) temperature=7>, <python_weather.forecast.DailyForecast date=datetime.date(2026, 1, 22) temperature=9>, <python_weather.forecast.DailyForecast date=datetime.date(2026, 1, 23) temperature=10>]
2026-01-21
7
[<python_weather.forecast.HourlyForecast time=datetime.time(0, 0) temperature=7 kind=Kind.LIGHT_SHOWERS>, <python_weather.forecast.HourlyForecast time=datetime.time(3, 0) temperature=7 kind=Kind.LIGHT_RAIN>, <python_weather.forecast.HourlyForecast time=datetime.time(6, 0) temperature=7 kind=Kind.LIGHT_SHOWERS>, <python_weather.forecast.HourlyForecast time=datetime.time(9, 0) temperature=7 kind=Kind.LIGHT_RAIN>, <python_weather.forecast.HourlyForecast time=datetime.time(12, 0) temperature=7 kind=Kind.LIGHT_RAIN>, <python_weather.forecast.HourlyForecast time=datetime.time(15, 0) temperature=9 kind=Kind.HEAVY_RAIN>, <python_weather.forecast.HourlyForecast time=datetime.time(18, 0) temperature=7 kind=Kin